In [2]:
# Import and create a new SQLContext 
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
# Read the country CSV file into an RDD.
country_lines = sc.textFile('file:///home/cloudera/Downloads/big-data-3/final-project/country-list.csv')
country_lines.take(5)

['Afghanistan, AFG',
 'Albania, ALB',
 'Algeria, ALG',
 'American Samoa, ASA',
 'Andorra, AND']

In [4]:
# Convert each line into a pair of words
country_tuples = country_lines.map(lambda pair : tuple(pair.split(",")))


In [5]:
# Convert each pair of words into a tuple


In [29]:
# Create the DataFrame, look at schema and contents
countryDF = sqlContext.createDataFrame(country_tuples, ["country", "code"])
countryDF.printSchema()
countryDF.show()

root
 |-- country: string (nullable = true)
 |-- code: string (nullable = true)

+-------------------+----+
|            country|code|
+-------------------+----+
|        Afghanistan| AFG|
|            Albania| ALB|
|            Algeria| ALG|
|     American Samoa| ASA|
|            Andorra| AND|
|             Angola| ANG|
|           Anguilla| AIA|
|Antigua and Barbuda| ATG|
|          Argentina| ARG|
|            Armenia| ARM|
|              Aruba| ARU|
|          Australia| AUS|
|            Austria| AUT|
|         Azerbaijan| AZE|
|            Bahamas| BAH|
|            Bahrain| BHR|
|         Bangladesh| BAN|
|           Barbados| BRB|
|            Belarus| BLR|
|            Belgium| BEL|
+-------------------+----+
only showing top 20 rows



In [19]:
# Read tweets CSV file into RDD of lines
tweet_lines = sc.textFile("file:///home/cloudera/Downloads/big-data-3/mongodb/tweet_data.csv")
tweet_lines.take(2)

['RT @ochocinco: I beat them all for 10 straight hours #FIFA16KING  https://t.co/BFnV6jfkBL',
 'RT @NiallOfficial: @Louis_Tomlinson @socceraid when I retired from playing because of my knee . I went and did my uefa A badges in Dublin']

In [21]:
# Clean the data: some tweets are empty. Remove the empty tweets using filter() 
tweet_lines.filter(lambda x: x != '')
tweet_lines.count()

13994

In [20]:
# Perform WordCount on the cleaned tweet texts. (note: this is several lines.)

word_list = tweet_lines.flatMap(lambda line: line.split(" "))

word_tuple = word_list.map(lambda word: (word, 1))
word_tuple = word_tuple.reduceByKey(lambda word1, word2: word1 + word2)

[('', 3896),
 ('https://t.co/fQftAwGAad', 1),
 ('mobile', 1),
 ('#FridayNightTouchdown', 1),
 ('circle', 7),
 ('#thfc', 2),
 ('reinstated', 4),
 ('like?"', 1),
 ('Bellow', 1),
 ('now"', 1),
 ('https://t.co/W4QluWGyeq', 1),
 ('https://t.co/qMkpvzgr0Y', 1),
 ('NINTENDO', 1),
 ('year-', 1),
 ('belt', 1),
 ('ago"', 1),
 ('laundry', 1),
 ('https://t.co/BLUXNc2wGB', 1),
 ('"BEST', 2),
 ('why', 130)]

In [55]:
# Create the DataFrame of tweet word counts
tweet_words = sqlContext.createDataFrame(word_tuple, ["tweet_word", "num"])
tweet_words.printSchema()
tweet_words.show()

root
 |-- tweet_word: string (nullable = true)
 |-- num: long (nullable = true)

+--------------------+----+
|          tweet_word| num|
+--------------------+----+
|                    |3896|
|https://t.co/fQft...|   1|
|              mobile|   1|
|#FridayNightTouch...|   1|
|              circle|   7|
|               #thfc|   2|
|          reinstated|   4|
|              like?"|   1|
|              Bellow|   1|
|                now"|   1|
|https://t.co/W4Ql...|   1|
|https://t.co/qMkp...|   1|
|            NINTENDO|   1|
|               year-|   1|
|                belt|   1|
|                ago"|   1|
|             laundry|   1|
|https://t.co/BLUX...|   1|
|               "BEST|   2|
|                 why| 130|
+--------------------+----+
only showing top 20 rows



In [56]:
# Join the country and tweet data frames (on the appropriate column)
inner_join = countryDF.join(tweet_words, countryDF.country == tweet_words.tweet_word)
inner_join.show()

+-----------+----+-----------+---+
|    country|code| tweet_word|num|
+-----------+----+-----------+---+
|   Thailand| THA|   Thailand|  1|
|    Iceland| ISL|    Iceland|  2|
|     Mexico| MEX|     Mexico|  1|
|      Wales| WAL|      Wales| 19|
|    Denmark| DEN|    Denmark|  1|
|      India| IND|      India|  4|
|   Portugal| POR|   Portugal|  8|
|     Poland| POL|     Poland|  1|
|     Norway| NOR|     Norway| 52|
|     Guinea| GUI|     Guinea|  8|
|   Slovakia| SVK|   Slovakia| 30|
|     Canada| CAN|     Canada| 11|
|Netherlands| NED|Netherlands| 13|
|      Kenya| KEN|      Kenya|  3|
|       Oman| OMA|       Oman|  1|
|      Qatar| QAT|      Qatar|  4|
|     Brazil| BRA|     Brazil| 13|
|    England| ENG|    England| 25|
|    Albania| ALB|    Albania|  1|
|  Argentina| ARG|  Argentina|  2|
+-----------+----+-----------+---+
only showing top 20 rows



In [58]:
inner_join = inner_join.select("country", "code", "num")
inner_join.show()

+-----------+----+---+
|    country|code|num|
+-----------+----+---+
|   Thailand| THA|  1|
|    Iceland| ISL|  2|
|     Mexico| MEX|  1|
|      Wales| WAL| 19|
|    Denmark| DEN|  1|
|      India| IND|  4|
|   Portugal| POR|  8|
|     Poland| POL|  1|
|     Norway| NOR| 52|
|     Guinea| GUI|  8|
|   Slovakia| SVK| 30|
|     Canada| CAN| 11|
|Netherlands| NED| 13|
|      Kenya| KEN|  3|
|       Oman| OMA|  1|
|      Qatar| QAT|  4|
|     Brazil| BRA| 13|
|    England| ENG| 25|
|    Albania| ALB|  1|
|  Argentina| ARG|  2|
+-----------+----+---+
only showing top 20 rows



In [59]:
# Question 1: number of distinct countries mentioned
inner_join.select("country").count()

44

In [60]:
# Question 2: number of countries mentioned in tweets.
from pyspark.sql.functions import sum
inner_join.select(sum("num")).show()

+--------+
|sum(num)|
+--------+
|     397|
+--------+



In [61]:
inner_join.describe("num").show()

+-------+-----------------+
|summary|              num|
+-------+-----------------+
|  count|               44|
|   mean|9.022727272727273|
| stddev|12.62977036076866|
|    min|                1|
|    max|               52|
+-------+-----------------+



In [63]:
# Question 3: what are the three countries with the highest mentioned count?
inner_join.orderBy(inner_join.num.desc()).show(3)

+-------+----+---+
|country|code|num|
+-------+----+---+
| Norway| NOR| 52|
|Nigeria| NGA| 49|
| France| FRA| 42|
+-------+----+---+
only showing top 3 rows



In [74]:
# Question 4: How many times was France mentioned in a tweet?
inner_join.filter(inner_join.country == "France").show()

+-------+----+---+
|country|code|num|
+-------+----+---+
| France| FRA| 42|
+-------+----+---+



In [77]:
# Question 5: which country was mentioned the most: Kenya, Wales, or Netherlands?
# inner_join.filter(inner_join.country == "Kenya").show()
inner_join.filter((inner_join.country == 'Kenya') | (inner_join.country == 'Wales') | (inner_join.country == 'Netherlands')).show()

+-----------+----+---+
|    country|code|num|
+-----------+----+---+
|      Wales| WAL| 19|
|Netherlands| NED| 13|
|      Kenya| KEN|  3|
+-----------+----+---+



In [80]:
# Question 6: what is the average number of times a country is mentioned?

inner_join.describe("num").show()

+-------+-----------------+
|summary|              num|
+-------+-----------------+
|  count|               44|
|   mean|9.022727272727273|
| stddev|12.62977036076866|
|    min|                1|
|    max|               52|
+-------+-----------------+



In [ ]:
# Table 1: top three countries and their counts.
from pyspark.sql.functions import desc


In [ ]:
# Table 2: counts for Wales, Iceland, and Japan.
